# Federated Learning - MNIST - Role: Data Scientist

This is the notebook to be developed by the *data scientist*. 
 * As a Data Scientist and you do not know where data lives, you only have access to GridNetwork.

**Goal: Train a remote Deep Learning model**

In this notebbok, we will show how to train a Federated Deep Learning with data hosted in Nodes.



## 0 - Previous setup

Components:

 - PyGrid Network      http://alice:7000
 - PyGrid Node Alice (http://bob:5000)
 - PyGrid Node Bob   (http://charlie:5001)

This tutorial assumes that these components are running in background. See [instructions](https://github.com/OpenMined/PyGrid/tree/dev/examples#how-to-run-this-tutorial) for more details.

### Import dependencies
Here we import core dependencies

In [1]:
import syft as sy
from syft.grid.public_grid import PublicGridNetwork

import torch as th

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import datasets, transforms


### Syft and client configuration
Now we hook Torch and connect to the GridNetwork. This is the only sever you do not need to know node addresses (networks knows), but lets first define some useful parameters

In [2]:
grid_address = "http://0.0.0.0:7000"  # address
N_EPOCHS = 100  # number of epochs to train
N_TEST   = 10   # number of test

In [3]:
hook = sy.TorchHook(th)


# Connect direcly to grid nodes
my_grid = PublicGridNetwork(hook, grid_address)

## 1 - Define our Neural Network Arquitecture

Now we will define a Deep Learning Network, feel free to write your own model!

In [4]:
class Arguments():
    def __init__(self):
        self.test_batch_size = N_TEST
        self.epochs = N_EPOCHS
        self.lr = 0.01
        self.log_interval = 5
        self.device = th.device("cpu")
        
args = Arguments()

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = Net()
model.to(args.device)

optimizer = optim.SGD(model.parameters(), lr=0.01)


## 2 - Search for remote data

Once we have defined our Deep Learning Network, we need some data to train... Thanks to PyGridNetwork this is very easy, you just need to search for your tags of interest.

Notice that _search()_ method  returns a pointer tensor, so we will work with those keeping real tensors hosted in Alice and Bob

In [6]:
data = my_grid.search("#X", "#mnist", "#dataset")  # images
target = my_grid.search("#Y", "#mnist", "#dataset")  # labels

data = list(data.values())  # returns a pointer
target = list(target.values())  # returns a pointer

If we print the tensors, we can check how the metadata we added before is included

In [7]:
print(data)
print(target)

[[(Wrapper)>[PointerTensor | me:84975565974 -> bob:61788994027]
	Tags: #X #dataset #mnist 
	Shape: torch.Size([5000, 1, 28, 28])
	Description: The input datapoints to the MNIST dataset....], [(Wrapper)>[PointerTensor | me:44971294991 -> alice:31469484730]
	Tags: #X #dataset #mnist 
	Shape: torch.Size([5000, 1, 28, 28])
	Description: The input datapoints to the MNIST dataset....]]
[[(Wrapper)>[PointerTensor | me:78421685917 -> bob:90201928853]
	Tags: #dataset #mnist #Y 
	Shape: torch.Size([5000])
	Description: The input labels to the MNIST dataset....], [(Wrapper)>[PointerTensor | me:38922590983 -> alice:3024702016]
	Tags: #dataset #mnist #Y 
	Shape: torch.Size([5000])
	Description: The input labels to the MNIST dataset....]]


## 3 - Train the model

Now we are ready to train. As you will see, this is very similar to standard pytorch sintax.

Let's first load test data in order to evaluate the model

In [8]:
transform = transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,)),  #  mean and std 
                              ])
testset = datasets.MNIST('./dataset', download=False, train=False, transform=transform)
testloader = th.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=True)

In [9]:
# epoch size
def epoch_total_size(data):
    total = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            total += data[i][j].shape[0]
            
    return total

In [10]:
def train(args):
    
    model.train()
    epoch_total = epoch_total_size(data)
    
    current_epoch_size = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            
            current_epoch_size += len(data[i][j])
            worker = data[i][j].location  # worker hosts data
            
            model.send(worker)  # send model to PyGridNode worker
            optimizer.zero_grad()  
            
            pred = model(data[i][j])
            loss = F.nll_loss(pred, target[i][j])
            loss.backward()
            
            optimizer.step()
            model.get()  # get back the model
            
            loss = loss.get()
            
        if epoch % args.log_interval == 0:

            print('Train Epoch: {} | With {} data |: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                      epoch, worker.id, current_epoch_size, epoch_total,
                            100. *  current_epoch_size / epoch_total, loss.item()))



In [11]:
def test(args):
    
    if epoch % args.log_interval == 0:
    
        model.eval()
        test_loss = 0
        correct = 0
        with th.no_grad():
            for data, target in testloader:
                data, target = data.to(args.device), target.to(args.device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
                pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(testloader.dataset)

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader.dataset),
            100. * correct / len(testloader.dataset)))

In [12]:
for epoch in range(N_EPOCHS):
    train(args)
    test(args)

Train Epoch: 0 | With bob data |: [5000/10000 (50%)]	Loss: 2.307125
Train Epoch: 0 | With alice data |: [10000/10000 (100%)]	Loss: 2.302680

Test set: Average loss: 2.2972, Accuracy: 914/10000 (9%)

Train Epoch: 5 | With bob data |: [5000/10000 (50%)]	Loss: 2.262557
Train Epoch: 5 | With alice data |: [10000/10000 (100%)]	Loss: 2.258535

Test set: Average loss: 2.2522, Accuracy: 3600/10000 (36%)

Train Epoch: 10 | With bob data |: [5000/10000 (50%)]	Loss: 2.216732
Train Epoch: 10 | With alice data |: [10000/10000 (100%)]	Loss: 2.212326

Test set: Average loss: 2.2045, Accuracy: 5554/10000 (56%)

Train Epoch: 15 | With bob data |: [5000/10000 (50%)]	Loss: 2.162978
Train Epoch: 15 | With alice data |: [10000/10000 (100%)]	Loss: 2.157516

Test set: Average loss: 2.1474, Accuracy: 6619/10000 (66%)

Train Epoch: 20 | With bob data |: [5000/10000 (50%)]	Loss: 2.093618
Train Epoch: 20 | With alice data |: [10000/10000 (100%)]	Loss: 2.086318

Test set: Average loss: 2.0727, Accuracy: 6958/1000

Et voilà! Here you are, you have trained a model on remote data using Federated Learning!

### References

This notebook is based on the reference example of [PyGrid Network Data-Centric](https://github.com/OpenMined/PyGrid)